In [ ]:
from dataloader.result_loader import KinectResultLoader, ArbeResultLoader, OptitrackResultLoader
import numpy as np
import os
root_path = "/home/nesc525/chen/3DSVC/__test__/default"

arbe_loader = ArbeResultLoader(root_path)
arbe_loader.file_dict['arbe'] = arbe_loader.file_dict['arbe'].drop_duplicates(subset=['dt'],keep='first')
ids = arbe_loader.file_dict['arbe']['id']
ids = sorted(map(int, ids))
print(ids)
arbe_loader[ids[0]]["arbe"]
os.listdir(root_path)
frames = range(0, 5, 1)
print(frames)
ids[frames]